In [42]:
import numpy as np
from scipy.special import gammaincc, gamma

In [43]:
# parameter
S0 = 1900
K = 2000
B = 2200
r = 0.0025
q = 0.015
T = 0.5
vol = 0.25
nuJ = 0.31
theta = -0.25
Y= 0.4

lp = np.sqrt(theta**2 / vol**4 + 2 / (nuJ * vol**2)) - theta / vol**2
ln = np.sqrt(theta**2 / vol**4 + 2 / (nuJ * vol**2)) + theta / vol**2

In [44]:
# quoted from Prof. Ali Hirsa sample code
def tridiagSolver(l, d, u, f):
    
    ''' 
    Tridiagonal solver 
    '''
    
    n = len(f)
    v = np.zeros(n)
    y = np.zeros(n)
    w = d[0]
    y[0] = 1.0 * f[0] / w
    
    for i in range(1, n):
        v[i-1] = 1. * u[i-1] / w
        w = d[i] - l[i] * v[i-1]
        y[i] = 1. * (f[i] - l[i] * y[i-1]) / w
    
    for j in range(n-2, -1, -1):
        y[j] = y[j] - v[j] * y[j+1]
    
    return y

In [45]:
xMin, xMax = np.log(500), np.log(B)
N = 800
M = 100

dx = (xMax - xMin) / N
dt = T / M

tau = dt * np.arange(1, M+1)
epsilon = dx
x = xMin + np.arange(N+1) * dx

In [46]:
# precalculated vector, since we know Y is not zero, we can simplify the expression of g1 and g2
def g1(x, alpha):
    return gammaincc(1-alpha, x) * gamma(1-alpha)

def g2(x, alpha):
    return ((np.exp(-x) * (x**(-alpha)) / alpha)) - g1(x, alpha) / alpha

def sig_calculator(l):
    return (l**(Y-2)) * (-(l*epsilon)**(1-Y) * np.exp(-l*epsilon) + (1-Y) * (g1(0, Y) - g1(l*epsilon, Y))) / nuJ 


kx = np.arange(1, N+1) * dx

# g vector 
g1n = g1(kx * ln, Y)
g1p = g1(kx * lp, Y)
g2n = g2(kx * ln, Y)
g2p = g2(kx * lp, Y)
g2n_plus = g2(kx * (ln+1), Y)
g2p_minus = g2(kx * (lp-1), Y)

# sigma and omega
sigma = sig_calculator(ln) + sig_calculator(lp)
omega = ((lp**Y) * g2(lp*epsilon, Y) - ((lp-1)**Y * g2((lp-1)*epsilon, Y)) \
+ (ln**Y) * g2(ln*epsilon, Y)  - ((ln+1)**Y * g2((ln+1)*epsilon, Y))) / nuJ

# diagonal entry
alpha = sigma * dt / (2 * dx**2)
beta = r - q + omega - (sigma / 2)

Bl = alpha - beta * dt / (2*dx)
Bu = alpha + beta * dt / (2*dx)


In [47]:
def R(w):
    ans = np.zeros(N-1)
    for i in range(1, N):
        if i == 1 or i == N-1:
            ans[i-1] = 0
        else:
            for k in range(1, i):
                ans[i-1] += ln**Y * (w[i-k] - w[i] - k * (w[i-k-1] - w[i-k])) * (g2n[k-1] - g2n[k])
                ans[i-1] += (w[i-k-1] - w[i-k]) * (g1n[k-1] - g1n[k]) / ((ln ** (1-Y)) * dx)

            for k in range(1, N-i):
                ans[i-1] += lp**Y * (w[i+k] - w[i] - k * (w[i+k+1] - w[i+k])) * (g2p[k-1] - g2p[k])
                ans[i-1] += (w[i+k-1] - w[i+k]) * (g1p[k-1] - g1p[k]) / ((lp ** (1-Y)) * dx)
        ans[i-1] += K * ln**Y * g2n[i-1] - np.exp(x[i]) * (ln + 1)**Y * g2n_plus[i-1] 
    return ans

In [48]:
# tridiagonal matrix
l = np.ones(N-1) * (-Bl)
u = np.ones(N-1) * (-Bu)
d = 1 + r*dt + Bu + Bl + dt * (ln**Y * g2n[:N-1] + lp**Y * g2p[::-1][:N-1]) / nuJ

# boundary conditon
u[-1] =  0
l[0] = 0

# initial condition
s = np.exp(x)
vCall = np.maximum(s - K, 0) * (s < B)

In [49]:
print('Implicit Scheme for UOC option')
for j in range(M):
    if (j+1) % 50  == 0:
        print('Iter = ' + str(j+1))   
    rhs = (dt * R(vCall) / nuJ) + vCall[1:N]
    inner = tridiagSolver(l, d, u, rhs)
    vCall = np.pad(inner, (1, 1), 'constant', constant_values=(0, 0))
print('Done')

Implicit Scheme for UOC option
Iter = 50
Iter = 100
Done


In [50]:
# can change M & N to test sensitivity 
uoc_imp = np.interp(np.log(S0), x, vCall)
print('Price of the UOC option through implicit scheme:', uoc_imp)

Price of the UOC option through implicit scheme: 33.41145860422138
